<a href="https://www.kaggle.com/code/nayanca12/final-project?scriptVersionId=210641272" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pytesseract
from ultralytics import YOLO
import cv2
import openpyxl
import os
import sqlite3
from ultralytics.utils.plotting import Annotator

In [ ]:
!pip install ultralytics
!pip install opencv-python

In [ ]:
!pip install pytesseract

In [ ]:
!rm /opt/conda/lib/libcurl.so.4 
!ln -s /usr/lib/x86_64-linux-gnu/libcurl.so.4.8.0 /opt/conda/lib/libcurl.so.4
!add-apt-repository ppa:alex-p/tesseract-ocr5 -y
!apt update
!apt install -y tesseract-ocr
!apt install tesseract-ocr-eng
# If you want all languages -- please note this will take time to download. You could also point to the already install tessdata in 4.00 as well.
# !apt install tesseract-ocr-all -y
!tesseract --version

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
# Function to create the table if it doesn't exist
def create_table():
    # Connect to the SQLite database
    conn = sqlite3.connect("numberplate.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Create the "vehicles" table with two columns: "vehicle_number" and "bike_image_path"
    # The "IF NOT EXISTS" clause ensures that the table is only created if it doesn't already exist
    c.execute("CREATE TABLE IF NOT EXISTS vehicles (vehicle_number TEXT, bike_image_path TEXT)")
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()

In [ ]:
# Function to insert a record into the "vehicles" table
def insert_record(vehicle_number, bike_image_path):
    # Connect to the SQLite database
    conn = sqlite3.connect("numberplate.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Insert the record into the "vehicles" table using parameterized SQL statement
    c.execute("INSERT INTO vehicles VALUES (?, ?)", (vehicle_number, bike_image_path))
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()

In [ ]:
# workbook = openpyxl.Workbook()
# worksheet = workbook.active

In [ ]:
# def write_to_excel(text, image_path):
#     worksheet.append([text, image_path])
#     workbook.save("C:/Users/PRAJWAL M/Registration_Plate_Recognition - Copy/Book1.xlsx")

In [ ]:
# # Initialize YOLO models
# person_bike_model = YOLO(r"C:/Users/PRAJWAL M/Desktop/main_model/300epoch.pt")
# helmet_model = YOLO(r"C:/Users/PRAJWAL M/Desktop/bike-helmet model/300epoch_bh.pt")
# number_plate_model = YOLO(r"C:/Users/PRAJWAL M/Desktop/vehicle model/300epochv.pt")

In [ ]:
# Initialize YOLO models
person_bike_model = YOLO("/kaggle/input/person_on_bike_model/pytorch/default/1/best.pt")
helmet_model = YOLO("/kaggle/input/helmet_model/pytorch/default/1/best.pt")
number_plate_model = YOLO("/kaggle/input/numberplate_model/pytorch/default/1/300epochv.pt")

In [ ]:
# Set up Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\nayan\OneDrive\Desktop\Tesseract_download\Tesseract-OCR\tesseract.exe"

In [ ]:
#Check OCR
image_path = "/kaggle/input/testtt/WhatsApp Image 2024-11-21 at 13.01.29_69035291.jpg"
frame = cv2.imread(image_path)
text = pytesseract.image_to_string(frame)
# cv2.imshow("window",frame)
print(text)

In [ ]:
image_path = "/kaggle/input/imagee"
# frame = cv2.imread(image_path)


In [ ]:
#tesseract ocr alternative
# import easyocr

# reader = easyocr.Reader(['en'])
# extract_info = reader.readtext(image_path)

# for el in extract_info:
#    print(el)

In [ ]:
# output_dir = r"C:/Users/PRAJWAL M/Registration_Plate_Recognition - Copy/output"  # Directory to save the output images for image folder
output_dir = "/kaggle/working/"  # Directory to save the output images for image folder

In [ ]:
output_dir1 = r"C:/Users/PRAJWAL M/Registration_Plate_Recognition - Copy/newoutput"  # Directory to save the output images for video input

In [ ]:
#model test
person_bike_results = person_bike_model.predict(image_path, show=True, save=True)
h_results = helmet_model.predict(image_path, show=True, save=True)
n_results = number_plate_model.predict(image_path, show=True, save=True)

In [ ]:
import os

# Path to the folder containing images
input_folder = "/kaggle/input/newdata/otest"


# Get a list of all image files in the input folder
image_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

# Process each image in the folder
for image_file in image_files:
    # Load the image
    img = cv2.imread(image_file)

    # Process frame
    #img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect person on a bike
    person_bike_results = person_bike_model.predict(img, show=True, save=True)
    
    # Process each detection result
    for r in person_bike_results:
        boxes = r.boxes
        # Filter detections for person on a bike
        for box in boxes:
            cls = box.cls
            print(person_bike_model.names[int(cls)], person_bike_model.names[int(cls)] == "person_bike")
            if person_bike_model.names[int(cls)] == "person_bike":
                # Crop person on a bike image
                x1, y1, x2, y2 = box.xyxy[0]
                person_bike_image = img[int(y1):int(y2), int(x1):int(x2)]

                # Detect helmet on the person
                helmet_results = helmet_model.predict(person_bike_image, show=True, save=True)

                # Process each helmet detection result
                for hr in helmet_results:
                    h_boxes = hr.boxes
                    # Filter detections for no helmet
                    for h_bo in h_boxes:
                        h_cls = h_bo.cls
                        if not helmet_model.names[int(h_cls)] == "With Helmet" :
                            # Extract number plate from the person bike image
                            number_plate_results = number_plate_model.predict(person_bike_image, show=True, save=True)

                            # Process each number plate detection result
                            for npr in number_plate_results:
                                np_boxes = npr.boxes
                                # Filter detections for number plate
                                for np_box in np_boxes:
                                    np_cls = np_box.cls
                                    print(number_plate_model.names[int(np_cls)])
                                    if number_plate_model.names[int(np_cls)] == "License_Plate":
                                        # Crop number plate image
                                        np_x1, np_y1, np_x2, np_y2 = np_box.xyxy[0]
                                        number_plate_image = person_bike_image[int(np_y1):int(np_y2),
                                                             int(np_x1):int(np_x2)]
                                        # Save the cropped images of persons on bikes with violations
                                        output_file = f"person_bike_violation_{os.path.basename(image_file)}"
                                        output_path = os.path.join(output_dir, output_file)
                                        cv2.imwrite(output_path, person_bike_image)

                                        # Save the cropped number plate image
                                        output_file_number_plate = f"number_plate_violation_{os.path.basename(image_file)}"
                                        output_path_number_plate = os.path.join(output_dir, output_file_number_plate)
                                        cv2.imwrite(output_path_number_plate, number_plate_image)
                                        gray = cv2.cvtColor(number_plate_image, cv2.COLOR_BGR2GRAY)
                                        text = pytesseract.image_to_string(gray)
                                        # Example usage
                                        # Create the "vehicles" table if it doesn't exist
                                        create_table()
                                        # Insert two records into the "vehicles" table
                                        insert_record(text, output_path)
                                        # Print the extracted text
                                        print("Number Plate Text:", text)

In [ ]:
# video_capture = cv2.VideoCapture("C:/Users/PRAJWAL M/Desktop/testing/VID_20240322_135640.mp4")  
video_capture = cv2.VideoCapture("C:/Users/PRAJWAL M/Desktop/Untitled video - Made with Clipchamp.mp4")
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    # Process frame
    #img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect person on a bike
    person_bike_results = person_bike_model.predict(frame)

    # Process each detection result
    for r in person_bike_results:
        boxes = r.boxes
        # Filter detections for person on a bike
        for box in boxes:
            cls = box.cls
            print(person_bike_model.names[int(cls)], person_bike_model.names[int(cls)] == "person_bike")
            if person_bike_model.names[int(cls)] == "person_bike":
                # Crop person on a bike image
                x1, y1, x2, y2 = box.xyxy[0]
                person_bike_image = frame[int(y1):int(y2), int(x1):int(x2)]

                # Detect helmet on the person
                helmet_results = helmet_model.predict(person_bike_image)

                # Process each helmet detection result
                for hr in helmet_results:
                    h_boxes = hr.boxes
                    # Filter detections for no helmet
                    for h_bo in h_boxes:
                        h_cls = h_bo.cls
                        if not helmet_model.names[int(h_cls)] == "With Helmet" :
                            # Extract number plate from the person bike image
                            number_plate_results = number_plate_model.predict(person_bike_image)

                            # Process each number plate detection result
                            for npr in number_plate_results:
                                np_boxes = npr.boxes
                                # Filter detections for number plate
                                for np_box in np_boxes:
                                    np_cls = np_box.cls
                                    print(number_plate_model.names[int(np_cls)])
                                    if number_plate_model.names[int(np_cls)] == "License_Plate":
                                        # Crop number plate image
                                        np_x1, np_y1, np_x2, np_y2 = np_box.xyxy[0]
                                        number_plate_image = person_bike_image[int(np_y1):int(np_y2),
                                                             int(np_x1):int(np_x2)]
                                        # Save the cropped number plate image
                                        #output_file = f"person_violation_{image_file}"
                                        output_file = f"person_bike_violation_{os.path.basename(image_file)}"
                                        output_path = os.path.join(output_dir1, output_file)
                                        cv2.imwrite(output_path, person_bike_image)

                                        # Perform OCR on the number plate image
                                        gray = cv2.cvtColor(number_plate_image, cv2.COLOR_BGR2GRAY)
                                        text = pytesseract.image_to_string(gray)
                                        # Example usage
                                        # Create the "vehicles" table if it doesn't exist
                                        create_table()
                                        # Insert two records into the "vehicles" table
                                        insert_record(text, output_path)
                                        # Print the extracted text
                                        print("Number Plate Text:", text)